## 009.Using Simphy to simulate trees


In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import sys, subprocess, time, dendropy, os, copy
import numpy as np
from sklearn import manifold, metrics, cluster, neighbors, decomposition, preprocessing
import treesignal

In [2]:
def read_tree(path):
    tree = dendropy.Tree.get(path = path, schema="newick", preserve_underscores=True)
    for leaf in tree.leaf_nodes(): # simphy default is to name as numbers
        leaf.taxon.label = "s" + leaf.taxon.label # 1_2_1 becomes s1_2_1
    return tree

def simphy(ntaxa=8, n_sp=4, nloci=2):
    ## limited to less than 1000 species trees and loci per sptree (just b/c of padding)
    tmp_dir = "/tmp/simphy"
    cli = "/home/leo/Academic/Biomath/Simulation/SimPhy/bin/simphy"
    # sb, ld, lb, lt are rate of transfers, births, etc. sp. su, st are pop size, tree height, subst rate
    cli += " -sb f:0.00001 -ld f:0.0000004 -lb f:0.00000041 -lt f:0.000005 -sp f:10000 -su f:0.0001 -st f:1000000"
    # sg is generation time, ls is minimum # of species per locus tree
    cli += " -sg f:1 -sl F:" + str(ntaxa) + " -ls 4 -o " + tmp_dir
    cli += " -rs " + str(n_sp) + " -rl F:" + str(nloci) + " "
    now_time = time.perf_counter()
    proc_run = subprocess.run(cli, shell=True, stdout=subprocess.PIPE, universal_newlines=True)
    now_time = time.perf_counter() - now_time
    print("simphy finished in " + str(now_time) + "secs")
    
    ## read g_treesXX.trees and s_tree.treesfiles from directories 1 ... n_sptrees
    ## (where XX is left-padded number of loci)
    
    if nloci < 10:    n_pad=1
    elif nloci < 100: n_pad=2
    else:             n_pad=3
    if n_sp < 10:    s_pad=1
    elif n_sp < 100: s_pad=2
    else:            s_pad=3
    sp_trees = list(range(n_sp)) # we know sizes beforehand, just fill with arbitrary numbers
    gn_trees = list(range(n_sp))
    for i in range(n_sp):
        spath = tmp_dir + "/" + str(i+1).zfill(s_pad) # this path will be used below
        sp_trees[i] = read_tree(spath + "/s_tree.trees") # dendropy read plus name change
        gn_trees[i] = list(range(nloci))
        for j in range(nloci): 
            gn_trees[i][j] = read_tree(spath + "/g_trees" + str(j+1).zfill(n_pad) + ".trees")
    proc_run = subprocess.run("rm -rf " + tmp_dir, shell=True, stdout=subprocess.PIPE, universal_newlines=True)
    return sp_trees, gn_trees   

In [10]:
st, gt = simphy(ntaxa=10, nloci=5, n_sp=2)
spstring = ""
for sptree in st:
    spstring += sptree.as_string("newick",suppress_edge_lengths=True).rstrip()
print (spstring)

simphy finished in 0.012640057000680827secs
(((s4,(s3,s2)),s1),(s7,((s8,(s10,(s5,s9))),s6)));((s1,(s4,(s3,s2))),((s9,s10),((s8,(s7,s5)),s6)));


In [11]:
sp2 = treesignal.lowlevel_randomise_trees_with_spr_string(spstring,n_copies=1,n_spr=1)
print (sp2)
sp2 = treesignal.lowlevel_randomise_trees_with_spr_string(sp2,n_copies=1,n_spr=1)
print (sp2)

((((((s9,s5),s10),s8),s6),s7),(((s2,s3),s4),s1)); (((((s5,s7),s8),s6),(s9,s10)),(((s2,s3),s4),s1)); ((((((s6,s7),s8),s10),s9),s5),(((s2,s3),s4),s1)); ((((((((s2,s3),s4),s1),s5),s7),s8),s6),(s9,s10));
((((((s5,s9),s10),s8),s6),s7),(((s3,s2),s4),s1)); (((((s7,s5),s8),s6),(s10,s9)),(((s3,s2),s4),s1)); ((((((s7,s6),s8),s10),s9),s5),(((s3,s2),s4),s1)); ((((((((s3,s2),s4),s1),s5),s7),s8),s6),(s10,s9)); (((((s3,s7),s2),s4),s1),(((s8,s5),s6),(s10,s9))); ((((((s7,s9),s6),s8),s10),s5),(((s3,s2),s4),s1)); (((((((((s3,s2),s4),s1),s5),s7),s8),s9),s10),s6);
